In [ ]:
from azure.identity import AzureCliCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, ChatCompletionAgent
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet
from dotenv import load_dotenv
import asyncio
import requests
import pandas as pd
from semantic_kernel.functions import kernel_function

load_dotenv()

model = os.getenv("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME")

project_client = AzureAIAgent.create_client(credential=AzureCliCredential(),
                           conn_str=os.getenv("PROJECT_CONNECTION_STRING")
)

class RecognizedRevenue:
    @kernel_function(description="Gets recognized revenue, invoice date, billing date, and due date for single or multiple companies.")
    def get_recognized_revenue(self, companyName: str):
        # Load CSV file
        df = pd.read_csv("recognizedRevenue.csv")
        companyName = companyName.strip().lower()

        if companyName == "all":
            # Return full records for all companies
            return df.to_dict(orient="records")
        
        # Match specific company
        matched_row = df[df["Company"].str.lower() == companyName]
        
        if not matched_row.empty:
            return matched_row.to_dict(orient="records")[0]
        else:
            return f"Company '{companyName}' not found in the data."


class HistoricalPayment:

    @kernel_function(description="Gets historical payment 'on-time' patterns.")
    def get_historical_payment_patterns(self, companyName:str):
        if companyName.lower() == "all":
            return {"Company A":"95%", "Company B":"85%", "Company C":"60%"}
        if companyName.lower() == "company a":
            return "95%"
        if companyName.lower() == "company b":
            return "85%"
        if companyName.lower() == "company c":
            return "60%"

class CashFlow:

    @kernel_function(description="Calculates cash flow.")
    def cash_flow_calculation(self, inflow:int, outflow:int, riskyCustomerRevenue:int):
        return inflow-outflow-riskyCustomerRevenue

class ShipmentInformation:

    @kernel_function(description="Checks if the stocks are available for the number of days passed.")
    def get_shipment_information(self, noOfDays:int):
        if noOfDays>=60:
            return f"Stocks for {noOfDays} is not available."
        else:
            return f"Stocks for {noOfDays} is available."


class InventoryInformation:

    @kernel_function(description="Gets inventory stock size.")
    def get_inventory_information(self):
        return "50000"
    

data_access_agent_definition = await project_client.agents.create_agent(
        model=model,
        name="Data_Access_Agent",
        instructions="""
        You are a specialized AI agent responsible for providing access to financial data upon request
        by other agents in the system. You hold complete and accurate data for on-time payment history, 
        revenue recognition, cash flow calculation, shipment information, and inventory information.

        - If recognized revenue data (including invoice, due, and payment dates) is required for one or more companies, 
        call 'RecognizedRevenue()'. If the user says "all", return data for all companies. Otherwise, return data for that specific company.

        - If historical payment 'on-time' pattern is required, call 'HistoricalPayment()'.

        - If inventory stock related inquiry comes, call 'InventoryInformation()'.

        - If inquiry checks stock availability for a certain number of days, call 'ShipmentInformation()'.

        """,
    )

DataAccessAgent = AzureAIAgent(
        client=project_client,
        definition=data_access_agent_definition,
        plugins=[RecognizedRevenue(), HistoricalPayment(), ShipmentInformation(), InventoryInformation()]
    )

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

service = AzureChatCompletion(
    api_key=os.getenv("AZURE_API_KEY"),
    endpoint=os.getenv("AZURE_ENDPOINT"),
    api_version=os.getenv("API_VERSION"),
    deployment_name=os.getenv("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"),
)

Finance_Agent = ChatCompletionAgent(
    service=service,
    name="FinanceAgent",
    instructions=(
        """
        You are the Finance Agent. Your responsibility is to respond to cash inflow and outflow related queries,
        as well as queries about recognized revenue details.

        For revenue and invoice-related data (including invoice date, due date, and payment date), call the DataAccessAgent
        using 'RecognizedRevenue()'.

        - If the user asks for data about all companies, pass 'all' as the company name and return the full table.
        - If the user asks for a specific company, return only that company's data.

        Note that the total cash outflow is $90,000. If you identify any payment delays or discrepancies, send a message to the FPA Agent.
        """
    ),
    plugins=[DataAccessAgent]
)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

user_input = input()

thread = project_client.agents.create_thread()


async def get_response_from_agent():
    response =  await Finance_Agent.get_response(
        messages = user_input,
    )
    
    return response
   
response = await get_response_from_agent()
print(response)